In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats


In [1]:
from src import soporte_abtesting as sa


In [4]:
df = pd.read_csv('marketing_AB.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [8]:
df.describe()

,Unnamed: 0,user id,total ads,most ads hour
count,588101.000000,5.881010e+05,588101.000000,588101.000000
mean,294050.000000,1.310692e+06,24.820876,14.469061
std,169770.279668,2.022260e+05,43.715181,4.834634
min,0.000000,9.000000e+05,1.000000,0.000000
25%,147025.000000,1.143190e+06,4.000000,11.000000
50%,294050.000000,1.313725e+06,13.000000,14.000000
75%,441075.000000,1.484088e+06,27.000000,18.000000
max,588100.000000,1.654483e+06,2065.000000,23.000000


In [8]:
#Check if the categorical variables have appropriate number of levels
df_cat = df[['test group', 'converted', 'most ads day', 'most ads hour']]
df_cat.nunique()

test group        2
converted         2
most ads day      7
most ads hour    24
dtype: int64

In [9]:
#Check if the categorical variables have appropriate levels
for i in df_cat.columns:
    print(i.upper(), ":", df_cat[i].unique())

TEST GROUP : ['ad' 'psa']
CONVERTED : [False  True]
MOST ADS DAY : ['Monday' 'Tuesday' 'Friday' 'Saturday' 'Wednesday' 'Sunday' 'Thursday']
MOST ADS HOUR : [20 22 18 10 14 13 19 11 12 16 21  3 23  4  8  0  2 15  1  6 17  7  9  5]


## Outlier Detection

In [9]:
def filter_outliers(data, whisker=1.5):
    """
    Identifies outliers in a dataset using the IQR method.

    Parameters:
        data (pd.Series): A pandas Series containing numerical data.
        whisker (float): The multiplier for the interquartile range (IQR). Default is 1.5.

    Returns:
        pd.Series: A boolean Series where True indicates an outlier.
    """
    # Calculate the first and third quartiles
    Q1 = data.quantile(0.25)  # 25th percentile
    Q3 = data.quantile(0.75)  # 75th percentile

    # Compute the interquartile range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for non-outliers
    lower_bound = Q1 - whisker * IQR
    upper_bound = Q3 + whisker * IQR

    # Identify outliers
    is_outlier = data[(data < lower_bound) | (data > upper_bound)]

    return is_outlier

In [10]:
outliers = filter_outliers(df['total ads'])
print(f"Total number of outliers : {len(outliers)}")
print(f"Total number of recors   : {len(df)}")
print(f"Percentage of outliers   : {len(outliers)/len(df)*100:.2f}%")

Total number of outliers : 52057
Total number of recors   : 588101
Percentage of outliers   : 8.85%
